In [1]:
import concurrent.futures
import json
import tmdbsimple as tmdb
import requests
import pandas as pd
tmdb.API_KEY = 'c954b2ff29e90a6d83535fbc73c5438f'
tmdb.REQUESTS_SESSION = requests.Session()

API_CONNECTIONS = 20

with open('AllMovies_bechdeltest.com.json', 'r') as jsonfile:
    data = jsonfile.read()
bechdelmovies = json.loads(data)

m_ids = []
m_pass = []
movies = []


In [2]:
def getMovie(bmov):
    imdb_id = 'tt'+str(bmov['imdbid'])
    try:
        tmdb_id = tmdb.Find(imdb_id).info(external_source = 'imdb_id')['movie_results'][0]['id']
        movie = tmdb.Movies(tmdb_id)
        req = movie.info()
        creds = movie.credits()
        #print(creds)
    except:
        return {}
    numCast = 4
    cred = []
    for i in range(numCast+1):
        try:
            cred.append(creds['cast'][i]['gender'])
        except IndexError:
            cred.append(0)

    passed = 1 if bmov['rating'] == 3 else 0
    
    ret = {"passed":passed, "imdbid":bmov['imdbid'], 'year': bmov['year'], "FirstLeadGender": cred[0], "SecondLeadGender": cred[1], "ThirdLeadGender": cred[2],"FourthLeadGender": cred[3], 'budget': req['budget'], 'popularity': req['popularity']}
    return ret
    

with concurrent.futures.ThreadPoolExecutor(max_workers=API_CONNECTIONS) as executor:
    future_to_url = (executor.submit(getMovie, bmov) for bmov in bechdelmovies)  #[0:1000] ##Limit here for testing, remove slice for final run
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            data = future.result()
        except Exception as exc:
            print("exception:", exc)
            data = False
        finally:
            if data:
                movies.append(data)
            print(str(len(movies)),end="\r")

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(movies)

In [ ]:
df.to_csv('out.csv', index=False) 